# Interactive TFX pipeline with eager execution

In [1]:
import os
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import trainer_pb2
from tfx.types.artifact_utils import get_split_uri
from tfx.utils.dsl_utils import tfrecord_input
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.components import ImportExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer


In [2]:
HISTORY_SIZE = 15
DATA_PATH = './../data'
PIPELINE_PATH = './../interactive_pipeline'
PIPELINE_NAME = 'timeseries_forecasting'
TFRECORD_PATH = os.path.join(DATA_PATH, 'dataset.tfrecord')
TRANSFORM_MODULE = 'transform.py'
TRAINER_MODULE = 'trainer.py'

## Preprocessed data

This is output of the preprocessing step and input to the TFX pipeline (*dataset.tfrecord* file).

In [3]:
def read_sample(sample):
    features = {
        'sample': tf.io.FixedLenFeature((), tf.string),
        'label': tf.io.FixedLenFeature((), tf.float32)
    }
    example = tf.io.parse_single_example(sample, features)
    sample = tf.io.parse_tensor(example['sample'], out_type=tf.float32)
    sample = tf.reshape(sample, (HISTORY_SIZE, 7))
    label = example['label']
    return sample, label


raw_dataset = tf.data.TFRecordDataset([TFRECORD_PATH])
parsed_dataset = raw_dataset.map(read_sample)
for sample in parsed_dataset.take(1):
    print(f'Label: {sample[1].numpy()}')
    print(f'Sample: {sample[0].numpy()}')


Label: 3.226189613342285
Sample: [[-6.00675344e-01 -2.92813921e+00 -3.43195381e+01 -5.93135774e-01
  -1.01801097e+00 -6.29491091e-01 -1.18535650e+00]
 [ 3.05245638e+00  9.82328653e-01 -1.18701212e-01  3.13687706e+00
   2.67739713e-01  1.11726248e+00  1.10528648e+00]
 [ 1.06512666e+00 -3.31788361e-01  1.04850546e-01  1.05234647e+00
  -3.45727175e-01  3.65980983e-01  1.28957236e+00]
 [ 1.64720982e-01 -5.53829968e-01  2.54766613e-01  1.81046978e-01
  -4.87393737e-01  2.75559354e+00 -8.39378238e-01]
 [ 1.10184097e+00 -3.48037452e-01  3.15068722e-01  1.07897258e+00
  -1.01801097e+00  3.78934115e-01  1.30182183e+00]
 [ 2.60339856e-01 -6.87013865e-01  6.72528595e-02  2.50809163e-01
   9.82457697e-02  4.00042921e-01  3.19962084e-01]
 [ 1.29542649e+00  1.82160899e-01  9.56450254e-02  1.31529009e+00
   9.74807680e-01 -7.23521173e-01  5.03703535e-01]
 [ 5.36852884e+00  8.89501095e-01 -1.09122723e-01  5.48459482e+00
   6.69971585e-01 -6.87540293e-01  1.48229671e+00]
 [ 1.64490592e+00 -5.75154305e-

## TFX Pipeline

In [4]:
tf.executing_eagerly()
context = InteractiveContext(pipeline_name=PIPELINE_NAME,
                             pipeline_root=PIPELINE_PATH)


### ExampleGen Component

In [5]:
output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
        ]))
examples = tfrecord_input(DATA_PATH)
example_gen = ImportExampleGen(input=examples,
                                output_config=output_config)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./../interactive_pipeline/ImportExampleGen/examples/1, id: 2)]
        ))

Output of the ExampleGen Component:

In [6]:
examples_dir = example_gen.outputs['examples']._artifacts[0].uri
examples_path = os.path.join(examples_dir, 'train/data_tfrecord-00000-of-00001.gz')
raw_dataset = tf.data.TFRecordDataset([examples_path], compression_type='GZIP')

for raw_record in raw_dataset.take(1):
    print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xda\x03\n\x11\n\x05label\x12\x08\x12\x06\n\x04\xad4\xfd?\n\xc4\x03\n\x06sample\x12\xb9\x03\n\xb6\x03\n\xb3\x03\x08\x01\x12\x08\x12\x02\x08\x0f\x12\x02\x08\x07"\xa4\x03r[C@\xe4y{?\x9f\x19\xf3\xbd\x98\xc2H@.\x15\x89>u\x02\x8f?\x07z\x8d?\x12V\x88?*\xe0\xa9\xbe\xe2\xbb\xd6=J\xb3\x86?\'\x03\xb1\xbe\xdca\xbb>\xb5\x10\xa5?\x9e\xac(>\xcd\xc7\r\xbf\xc5p\x82>ad9>\xac\x8b\xf9\xbe\xa5[0@~\xe1V\xbf \t\x8d?\xf71\xb2\xbe\xb0P\xa1>\xc6\x1b\x8a?/N\x82\xbf\xa7\x03\xc2>\x19\xa2\xa6?DK\x85>$\xe0/\xbf\xde\xbb\x89=\x0fj\x80>\x145\xc9=m\xd2\xcc>\x12\xd2\xa3>\x89\xd0\xa5?c\x88:>\x8a\xe1\xc3=m[\xa8?\xff\x8cy?\xaf89\xbf\xb7\xf2\x00?\xfd\xca\xab@X\xb6c?\xbc{\xdf\xbd\xcd\x81\xaf@B\x83+?\xa4\x020\xbf\xe6\xbb\xbd?G\x8c\xd2?P=\x13\xbf\xd4\x82F>\x07\x9a\xe1?$\xceq=!\xf8\xc5?\xdd\x84&\xbf\xc8\xf1\xfb?\xe3 \xa3?>?\x07?]\xce\xf8?\xac\xccJ@!\x19L=\xfb\xe4r\xbf\xd3\xa0}@\x9e1\x87\xbe\x98\x1c:>e\x10{@\x90\x90\xa9\xbeP\xc4\xa5>\x99\x8c\xc6?\xb4\t\x86\xbd\x9ak\x96<\xfc\x9d\x19?\

In [7]:
features_description = {
    'sample': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.float32)
}


def parse_example(raw_example):
    return tf.io.parse_single_example(raw_example, features_description)


parsed_dataset = raw_dataset.map(parse_example)

for parsed_record in parsed_dataset.take(1):
    print(tf.io.parse_tensor(parsed_record['sample'], out_type=tf.float32))
    print(parsed_record['label'])

tf.Tensor(
[[ 3.0524564   0.98232865 -0.11870121  3.136877    0.2677397   1.1172625
   1.1052865 ]
 [ 1.0651267  -0.33178836  0.10485055  1.0523465  -0.34572718  0.36598098
   1.2895724 ]
 [ 0.16472098 -0.55382997  0.2547666   0.18104698 -0.48739374  2.7555935
  -0.83937824]
 [ 1.101841   -0.34803745  0.31506872  1.0789726  -1.018011    0.37893412
   1.3018218 ]
 [ 0.26033986 -0.68701386  0.06725286  0.25080916  0.09824577  0.40004292
   0.31996208]
 [ 1.2954265   0.1821609   0.09564503  1.3152901   0.9748077  -0.7235212
   0.50370353]
 [ 5.368529    0.8895011  -0.10912272  5.484595    0.6699716  -0.6875403
   1.4822967 ]
 [ 1.6449059  -0.5751543   0.19385844  1.762513    0.05903448  1.5466348
  -0.65046483]
 [ 1.9683161   1.2744411   0.52830875  1.9437977   3.1687422   0.04982865
  -0.94880646]
 [ 3.962941   -0.26405042  0.1817497   3.9228756  -0.33118105  0.32376337
   1.5511657 ]
 [-0.06544819  0.01836186  0.6000669  -0.0572128  -1.018011   -0.74079204
  -0.627327  ]
 [ 0.8454179   

### StatisticsGen Component

In [8]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

context.run(statistics_gen)

t be decoded as a UTF-8 string.


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: ./../interactive_pipeline/StatisticsGen/statistics/2, id: 3)]
        ))

Output of the StatisticsGen Component:

In [9]:
artifact_list = statistics_gen.outputs['output'].get()
train_artifacts_path = get_split_uri(artifact_list, 'train')
train_statistics_path = os.path.join(train_artifacts_path, 'stats_tfrecord')
eval_artifacts_path = get_split_uri(artifact_list, 'eval')
eval_statistics_path = os.path.join(eval_artifacts_path, 'stats_tfrecord')

In [10]:
train_statistics = tfdv.load_statistics(train_statistics_path)
eval_statistics = tfdv.load_statistics(eval_statistics_path)

tfdv.visualize_statistics(lhs_statistics=eval_statistics,
                          rhs_statistics=train_statistics,
                          lhs_name='Eval',
                          rhs_name='Train')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### SchemaGen Component

In [11]:
schema_gen = schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                                    infer_feature_shape=True)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ./../interactive_pipeline/SchemaGen/schema/3, id: 4)]
        ))

Output of the SchemaGen Component:

In [12]:
schema_directory = schema_gen.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_directory, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'label',FLOAT,required,,-
'sample',BYTES,required,,-


### ExampleValidator Component

In [13]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                     schema=schema_gen.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: ./../interactive_pipeline/ExampleValidator/anomalies/4, id: 5)]
        ))

Output of the ExampleValidator Component:

In [14]:
validator_directory = example_validator.outputs['output'].get()[0].uri
validator_path = os.path.join(validator_directory, 'anomalies.pbtxt')

anomalies = tfdv.load_anomalies_text(validator_path)
tfdv.display_anomalies(anomalies)

### Transform Component

In [15]:
transform = Transform(examples=example_gen.outputs['examples'],
                      schema=schema_gen.outputs['schema'],
                      module_file=TRANSFORM_MODULE)

context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./../interactive_pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/3f482ca0112e4b2095244cdc9e08f4a8/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ./../interactive_pipeline/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./../interactive_pipeline/Transform/transformed_examples/5, id: 7)]
        ))

Output of the Transform Component:

In [16]:
transform_directory = transform.outputs['transformed_examples'].get()[0].uri
transform_path = os.path.join(transform_directory, 'train', 'transformed_examples-00000-of-00001.gz')

transformed_dataset = tf.data.TFRecordDataset([transform_path], compression_type='GZIP')

for transformed_record in transformed_dataset.take(1):
    print(repr(transformed_record))

<tf.Tensor: shape=(), dtype=string, numpy=b"\n\xcb\x03\n\x11\n\x05label\x12\x08\x12\x06\n\x04\xad4\xfd?\n\xb5\x03\n\x06sample\x12\xaa\x03\x12\xa7\x03\n\xa4\x03r[C@\xe4y{?\x9f\x19\xf3\xbd\x98\xc2H@.\x15\x89>u\x02\x8f?\x07z\x8d?\x12V\x88?*\xe0\xa9\xbe\xe2\xbb\xd6=J\xb3\x86?'\x03\xb1\xbe\xdca\xbb>\xb5\x10\xa5?\x9e\xac(>\xcd\xc7\r\xbf\xc5p\x82>ad9>\xac\x8b\xf9\xbe\xa5[0@~\xe1V\xbf \t\x8d?\xf71\xb2\xbe\xb0P\xa1>\xc6\x1b\x8a?/N\x82\xbf\xa7\x03\xc2>\x19\xa2\xa6?DK\x85>$\xe0/\xbf\xde\xbb\x89=\x0fj\x80>\x145\xc9=m\xd2\xcc>\x12\xd2\xa3>\x89\xd0\xa5?c\x88:>\x8a\xe1\xc3=m[\xa8?\xff\x8cy?\xaf89\xbf\xb7\xf2\x00?\xfd\xca\xab@X\xb6c?\xbc{\xdf\xbd\xcd\x81\xaf@B\x83+?\xa4\x020\xbf\xe6\xbb\xbd?G\x8c\xd2?P=\x13\xbf\xd4\x82F>\x07\x9a\xe1?$\xceq=!\xf8\xc5?\xdd\x84&\xbf\xc8\xf1\xfb?\xe3 \xa3?>?\x07?]\xce\xf8?\xac\xccJ@!\x19L=\xfb\xe4r\xbf\xd3\xa0}@\x9e1\x87\xbe\x98\x1c:>e\x10{@\x90\x90\xa9\xbeP\xc4\xa5>\x99\x8c\xc6?\xb4\t\x86\xbd\x9ak\x96<\xfc\x9d\x19?\xf7Wj\xbd/N\x82\xbf\x8c\xa4=\xbf\x81\x98 \xbfOmX?\xa3\xb

In [17]:
transformed_features_description = {
    'sample': tf.io.FixedLenFeature([HISTORY_SIZE, 7], tf.float32),
    'label': tf.io.FixedLenFeature([], tf.float32)
}


def parse_transformed_example(example):
    return tf.io.parse_single_example(example, transformed_features_description)


parsed_dataset = transformed_dataset.map(parse_transformed_example)

for parsed_record in parsed_dataset.take(1):
    print(parsed_record['sample'])
    print(parsed_record['label'])

tf.Tensor(
[[ 3.0524564   0.98232865 -0.11870121  3.136877    0.2677397   1.1172625
   1.1052865 ]
 [ 1.0651267  -0.33178836  0.10485055  1.0523465  -0.34572718  0.36598098
   1.2895724 ]
 [ 0.16472098 -0.55382997  0.2547666   0.18104698 -0.48739374  2.7555935
  -0.83937824]
 [ 1.101841   -0.34803745  0.31506872  1.0789726  -1.018011    0.37893412
   1.3018218 ]
 [ 0.26033986 -0.68701386  0.06725286  0.25080916  0.09824577  0.40004292
   0.31996208]
 [ 1.2954265   0.1821609   0.09564503  1.3152901   0.9748077  -0.7235212
   0.50370353]
 [ 5.368529    0.8895011  -0.10912272  5.484595    0.6699716  -0.6875403
   1.4822967 ]
 [ 1.6449059  -0.5751543   0.19385844  1.762513    0.05903448  1.5466348
  -0.65046483]
 [ 1.9683161   1.2744411   0.52830875  1.9437977   3.1687422   0.04982865
  -0.94880646]
 [ 3.962941   -0.26405042  0.1817497   3.9228756  -0.33118105  0.32376337
   1.5511657 ]
 [-0.06544819  0.01836186  0.6000669  -0.0572128  -1.018011   -0.74079204
  -0.627327  ]
 [ 0.8454179   

### Trainer Component

In [30]:
trainer = Trainer(module_file=TRAINER_MODULE,
                  examples=transform.outputs['transformed_examples'],
                  schema=schema_gen.outputs['schema'],
                  transform_graph=transform.outputs['transform_graph'],
                  custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
                  train_args=trainer_pb2.TrainArgs(num_steps=35),
                  eval_args=trainer_pb2.EvalArgs(num_steps=8))

context.run(trainer)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 32)                5120      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total params: 5,153
Trainable params: 5,153
Non-trainable params: 0
_________________________________________________________________
Train for 35 steps, validate for 8 steps
Epoch 1/20
35/35 [==============================] - 1s 42ms/step - loss: 0.5555 - val_loss: 0.5420
Epoch 2/20
35/35 [==============================] - 0s 5ms/step - loss: 0.5268 - val_loss: 0.5605
Epoch 3/20
35/35 [==============================] - 0s 5ms/step - loss: 0.5165 - val_loss: 0.5475
Epoch 4/20
35/35 [==============================] - 0s 5ms/step - loss: 0.5107 - val_loss: 0.5335
Epoch 5/20
35/35 [==============================] - 0s 5ms/step - loss: 0.5053 

ExecutionResult(
    component_id: Trainer
    execution_id: 18
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ./../interactive_pipeline/Trainer/model/18, id: 20)]
        ))